# Scientific Paper Recommendation System with RAG
## Project Description
This project implements a Retrieval Augmented Generation (RAG) system for scientific paper recommendations. The system allows users to input a document or query and receive recommendations for relevant scientific papers from the ArXiv database.

## Key Components
Vector Database: Uses a ChromaDB vector database containing embeddings of 10,000 ArXiv research papers
Document Processing: Extracts and processes PDF content using PyMuPDF
Semantic Search: Performs similarity searches based on document content

## GenAI Functionalities
- Embeddings: Generates semantic embeddings using Google's text-embedding-004 model
- Prompt Engineering: Utilizes carefully crafted prompts to guide the AI's behavior
- RAG Implementation: Combines vector search results with generative AI responses
- Document Understanding: Processes and interprets PDF research papers
- Vector Embedding and Vector Search: Performs semantic similarity searches in high-dimensional vector space

The system orchestrates these components through a chatbot interface that processes user queries, searches for relevant papers, and generates comprehensive responses that include paper details like authors and publication dates.

## Database
ArXiv serves as an excellent data source for our recommendation system for several key reasons:

- **Rich Scientific Content**: Contains over 2 million scholarly articles across multiple disciplines
- **Well-Structured Metadata**: Includes titles, abstracts, authors, and categories in a consistent format
- **Embedding-Friendly**: Abstracts provide concise, information-dense text that produces meaningful vector embeddings
- **Research Relevance**: Widely used by the scientific community, ensuring practical utility
- **Semantic Search Compatibility**: Content structure works effectively with our embedding model (text-embedding-004)

Our implementation uses 10,000 ArXiv papers converted to vector embeddings and stored in ChromaDB, enabling semantic similarity searches to retrieve relevant scientific literature for user queries. The Arxiv dataset is available in Kagggle with the following link: [Arxiv_dataset](https://www.kaggle.com/datasets/Cornell-University/arxiv). 
## Use Case
Researchers can upload a scientific paper and ask questions to find related work in the ArXiv database, facilitating literature reviews and discovery of relevant research.

````
chatbot = RAG_Scientific_chatbot()
answer = chatbot.chat("Find me related papers", "/path/to/document.pdf")
display(Markdown(answer))
```` 


## Setup
Import and install the necessary libraries.

In [1]:
!pip uninstall -qqy jupyterlab kfp  # Remove unused conflicting packages
!pip install -qU "google-genai==1.7.0" "chromadb==0.6.3"
!pip install --upgrade pymupdf

from google import genai
from google.genai import types

import json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 16.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 47.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 73.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.

In [2]:
# API keys
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
client = genai.Client(api_key=GOOGLE_API_KEY)

In [3]:
# Define a retry policy. The model might make multiple consecutive calls automatically
# for a complex query, this ensures the client retries if it hits quota limits.
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

## Upload of Arxiv papers
First the arxiv dataset is imported and a vector embedding of all the documents is perfomed. After the vector embedding,these are stored in a chromadb vector database. The arxiv dataset import is shown below. Given that it is computationally expensive to import and process all papers, only a few are imported randomly.

In [4]:
import random
import numpy as np
amount_papers = 10000
papers = []

with open('/kaggle/input/arxiv/arxiv-metadata-oai-snapshot.json', 'r') as file:
    for i, line in enumerate(file):
        papers.append(json.loads(line))

random_indices = set(random.sample(range(len(papers)), amount_papers))
random_indices = list(random_indices)
papers_random = []
for i in range(len(random_indices)):
    index = random_indices[i]
    papers_random.append(papers[index])
papers = papers_random
# Now data is a list of dictionaries
print("Headers:", list(papers[0].keys()))


Headers: ['id', 'submitter', 'authors', 'title', 'comments', 'journal-ref', 'doi', 'report-no', 'categories', 'license', 'abstract', 'versions', 'update_date', 'authors_parsed']


Only the title and the abstract of each paper are embedded. The code below implements this preprocessing of the papers.

In [5]:
def remove_newlines(obj):
    if isinstance(obj, str):
        return obj.replace('\n', ' ')
        
preprocessed_papers = []
for paper in papers:
    preprocessed_papers.append("PAPER TITLE: " + remove_newlines(paper["title"]) + "\nPAPER CONTENT: "+ remove_newlines(paper["abstract"]))
print("SUCCESSFULLY PREPROCESSED "+ str(len(preprocessed_papers)) + " PAPERS")
print("--- EXAMPLE OF PREPROCESSED PAPER ---")
print(preprocessed_papers[0])

SUCCESSFULLY PREPROCESSED 10000 PAPERS
--- EXAMPLE OF PREPROCESSED PAPER ---
PAPER TITLE: Harmonic forcing of an extended oscillatory system: Homogeneous and   periodic solutions
PAPER CONTENT:   In this paper we study the effect of external harmonic forcing on a one-dimensional oscillatory system described by the complex Ginzburg-Landau equation (CGLE). For a sufficiently large forcing amplitude, a homogeneous state with no spatial structure is observed. The state becomes unstable to a spatially periodic ``stripe'' state via a supercritical bifurcation as the forcing amplitude decreases. An approximate phase equation is derived, and an analytic solution for the stripe state is obtained, through which the asymmetric behavior of the stability border of the state is explained. The phase equation, in particular the analytic solution, is found to be very useful in understanding the stability borders of the homogeneous and stripe states of the forced CGLE. 


Now the preprocessed papers are transformed into vector embeddings with the embedding model  models/text-embedding-004.

In [6]:
def batch(iterable, n=100):
    for i in range(0, len(iterable), n):
        yield iterable[i:i + n]

papers_embedded = []  
papers_batches = list(batch(preprocessed_papers, 100)) #limit of 100 embeddings per call

for batch in papers_batches:
    batch_embedded = client.models.embed_content(
        model='models/text-embedding-004',
        contents=batch,
        config=types.EmbedContentConfig(task_type='SEMANTIC_SIMILARITY'))
    list_batch_embedded = [e.values for e in batch_embedded.embeddings]
    papers_embedded+=list_batch_embedded

print("SUCCESSFULLY EMBEDDED "+ str(len(papers_embedded)) + " PAPERS")

SUCCESSFULLY EMBEDDED 10000 PAPERS


Once the vector embeddings of the papers are computed, these are stored into the chromadb database.

In [7]:
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
def batch(iterable, batch_size):
    for i in range(0, len(iterable), batch_size):
        yield iterable[i:i + batch_size]


# Start ChromaDB client
chromadb_client = chromadb.Client()

# Create or get a collection
collection = chromadb_client.get_or_create_collection(name="papers")

# Add the documents + embeddings to Chroma
emb_batches = list(batch(papers_embedded, 41000))
papers_batches = list(batch(preprocessed_papers, 41000))
for i in range(len(emb_batches)):
    ids_batch = [f"doc_{j + i * 41000}" for j in range(len(emb_batches[i]))]
    collection.add(
        documents=papers_batches[i],
        embeddings=emb_batches[i],
        ids=ids_batch,
    )
print("SUCCESSFULLY UPLOADED "+ str(len(papers_embedded)) + " PAPERS")

SUCCESSFULLY UPLOADED 10000 PAPERS


## Vector database search example

Now an extract of a sample paper is used to search for similar papers in the database. If the same paper is obtained, the queried paper was in the database.

In [8]:
query_input = "Statistical modeling of experimental physical laws is based on the probability density function of measured variables. It is expressed by experimental data via a kernel estimator. The kernel is determined objectively by the scattering of data during calibration of experimental setup. A physical law, which relates measured variables, is optimally extracted from experimental data by the conditional average estimator. It is derived directly from the kernel estimator and corresponds to a general nonparametric regression. T"
#query_input = pdf_text

query_embedding = client.models.embed_content(
        model='models/text-embedding-004',
        contents=query_input,
        config=types.EmbedContentConfig(task_type='SEMANTIC_SIMILARITY'))

In [9]:
results = collection.query(
    query_embeddings=[query_embedding.embeddings[0].values],
    n_results=5  # Number of similar docs to return
)

for doc, doc_id in zip(results["documents"][0], results["ids"][0]):
    print(f"ID: {doc_id}")
    print(f"{doc}\n")

ID: doc_6323
PAPER TITLE: Machine learning memory kernels as closure for non-Markovian stochastic   processes
PAPER CONTENT:   Finding the dynamical law of observable quantities lies at the core of physics. Within the particular field of statistical mechanics, the generalized Langevin equation (GLE) comprises a general model for the evolution of observables covering a great deal of physical systems with many degrees of freedom and an inherently stochastic nature. Although formally exact, the GLE brings its own great challenges. It depends on the complete history of the observables under scrutiny, as well as the microscopic degrees of freedom, all of which are often inaccessible. We show that these drawbacks can be overcome by adopting elements of machine learning from empirical data, in particular coupling a multilayer perceptron (MLP) with the formal structure of the GLE and calibrating the MLP with the data. This yields a powerful computational tool capable of describing noisy comple

## RETRIEVAL AUGMENTED GENERATION (RAG)
For retrieval augmented generation, the question of the user together with the document are used to search for useful papers. With the useful papers and the user input, an answer is generated. The steps are as follow:
1) Use a LLM to embed the user input question and input document for vector search.
2) Obtain the original documents from the vector search in the database.
3) Use the input question and input document and the original documents from the database to generate a response with a LLM.
4) Show the answer to the user.

Prompt engineering is employed to adjust the answers of the chatbot.

### Orchestration functions
The following functions orchestrate the RAG and found in the class RAG_Scientific_chatbot:

- _create_embedding(text): For a given text generates the corresponding vector embedding.
- _search_embedded_documents(query_embedding, n): For a given vector, searches nearby vectors in the vector embeddings database.
- _retrieve_documents(doc_id): For a given list of document ids, this function returns an extended information of each paper.

In [47]:
import pymupdf
import re

class RAG_Scientific_chatbot:
        
    def chat(self, question:str, document_path:str=None):
                           
        processed_input, user_document = self._process_input(question, document_path)
    
        embedding = self._create_embedding(processed_input["embedding_query"])
        
        search_output = self._search_embedded_documents(embedding, int(processed_input["num_documents"]))
        
        doc_ids = search_output['ids'][0]  
        numeric_ids = [int(doc.split('_')[1]) for doc in doc_ids]
        extended_info = self._retrieve_documents(numeric_ids)
        answer = self._generate_final_answer(question, user_document, search_output, extended_info)

        return answer


    # === Tools ===
    def _process_input(self, user_message: str, document_path:str = None ):
        instruction = """
            You are a scientific research assistant specializing in analyzing academic papers and research questions.
            
            Your task is to analyze the user's question and their uploaded document to create:
            1. An optimal embedding query for retrieving the most relevant scientific papers
            2. A recommendation for how many papers to retrieve
            
            INSTRUCTIONS:
            - Identify key scientific concepts, methodologies, domain-specific terminology, and research areas
            - Extract specific technical terms that would appear in related papers
            - Consider both the user's explicit question and the implicit research goals from their document
            - Focus on scientific significance rather than general terms
            - For empirical research questions, include methodology terms and measurement concepts
            - For theoretical questions, include relevant frameworks and paradigms
            
            For number of documents:
            - Suggest 3-5 papers for focused questions with specific methodology/technology
            - Suggest 5-8 papers for broader research areas requiring multiple perspectives
            - Suggest 8-12 papers for literature reviews or comparative analyses
            - If the amount of papers is explicit in QUESTION, search for this amount of papers or more
            
            
            Return this string output:
            "{"embedding_query": "<your optimized embedding query>",
             "num_documents": "<number of papers to retrieve>"}"
            
            EXAMPLES:
            
            Question: Find me 3 papers about science and 2 about nature.  
            Good num_documents: 7
            Bad num_documents: 3
            
            """

        if document_path != None:
            pdf_text = self._extract_text_from_pdf(document_path)
            contents = [
                types.Content(
                    role="user", parts=[types.Part(text=user_message),types.Part(text=pdf_text)]
                )
            ]
          
        else: 
            pdf_text = None
            contents = [
                types.Content(
                    role="user", parts=[types.Part(text=user_message)]
                )
            ]
        
        
        processed_input = client.models.generate_content(
            model="gemini-2.0-flash", 
            config=types.GenerateContentConfig(
                system_instruction=instruction,
            ),
            contents = contents
        )
        
        match = re.search(r'\{.*\}', processed_input.text, re.DOTALL)
        if match:
            clean_json_str = match.group(0)
            processed_input = json.loads(clean_json_str)
     
        return processed_input, pdf_text

    
    def _extract_text_from_pdf(self, path):
        text = ""
        with pymupdf.open(path) as doc:
            for page in doc:
                text += page.get_text()
        return text
        
        
    def _create_embedding(self, text)-> list:
        print(f' - CALL: create_embedding({text[:20]}...)')
        vector_embedding = client.models.embed_content(
            model='models/text-embedding-004',
            contents=text,
            config=types.EmbedContentConfig(task_type='SEMANTIC_SIMILARITY')
        )
        return vector_embedding.embeddings[0].values
    
    def _search_embedded_documents(self, query_embedding:list[float], n:int)->list[str]:
        print(f' - CALL: search_embedded_documents(n = {n})')
        results = collection.query(
            query_embeddings=[query_embedding],
            n_results=n
        )
        return results
    
    def _retrieve_documents(self, doc_ids:list[int])-> list[dict]:
        print(f' - CALL: retrieve_documents(IDS = {doc_ids})')
        papers_retrieved = []
        for doc_id in doc_ids:
            papers_retrieved.append(papers[doc_id])
            
        return papers_retrieved
        
    def _generate_final_answer(self,question: str, user_document: str, search_output: str, extended_info: str):
        instruction = """
            You are an advanced scientific research assistant tasked with providing comprehensive answers based on retrieved academic papers.
            
            CONTEXT:
            - The user has asked a QUESTION about a scientific topic
            - They've provided their own INPUT_DOCUMENT (a scientific paper or research proposal)
            - You've retrieved relevant papers from a scientific database (EMBED_DATA and EXTENDED_PAPER_INFO)
            
            YOUR TASK:
            1. Analyze the retrieved papers and determine their relevance to the question
            2. Provide an answer based on the retreived papers from EMBED_DATA and EXTENDED_PAPER_INFO. 
            3. You may use information from the INPUT_DOCUMENT if the papers from EMBED_DATA and EXTENDED_PAPER_INFO are not relevant. Always mention where the information is obtained from.
            
            IMPORTANT GUIDELINES:
            - SKIP the user's own paper if it appears in the results
            - Prioritize recent papers and high-impact findings
            - Compare and contrast contradictory findings when present
            - Always provide authors and publication dates
            - Focus on scientific significance rather than general summaries
            - For methodology questions, emphasize technical details and implementation
            - Use objective, academically-appropriate language
            - Provide the answer in a Markdown format
            - You do not need to show all the papers from EMBED_DATA and EXTENDED_PAPER_INFO, only the most relevant and important
            - For the answer, use only papers from the database. You may include some suggestions to other papers but don't make it too extensive.
            - If the numer of papers is explicit in the QUESTION, show only the amount asked
            - Do not mention the amount of papers retrieved from EMBED_DATA and EXTENDED_PAPER_INFO
            """

        if user_document != None:
            prompt =f"""
                    QUESTION:{user_message}
                    EMBED_DATA: {search_output}
                    EXTENDED_PAPER_INFO: {extended_info}
                    """
        else: 
            prompt =f"""
                    QUESTION:{user_message}
                    INPUT_DOCUMENT:{user_document}
                    EMBED_DATA: {search_output}
                    EXTENDED_PAPER_INFO: {extended_info}
                    """
        
        contents = []
        contents.append(types.Content(role="user", parts=[types.Part(text = prompt)]))
        response_final = client.models.generate_content(
            model="gemini-2.0-flash", 
            config=types.GenerateContentConfig(
                system_instruction=instruction
            ),
            contents = contents
        )
        return response_final.text

## Example Usage 1
Find scientific papers related to an input paper.

In [48]:
from IPython.display import display, Markdown, Latex


document = "/kaggle/input/unc-paper/2409.10655v2.pdf"
user_message = "Find me related papers with emphasis in uncertainty estimation."

chatbot = RAG_Scientific_chatbot()
answer = chatbot.chat(user_message, document)
display(Markdown(answer))

 - CALL: create_embedding(Deep Reinforcement L...)
 - CALL: search_embedded_documents(n = 8)
 - CALL: retrieve_documents(IDS = [3972, 8390, 5911, 1186, 2009, 7951, 9861, 4252])


Here are some papers that focus on uncertainty estimation in the context of reinforcement learning and decision-making:

1.  **Active operator learning with predictive uncertainty quantification for partial differential equations** by Winovich et al. (2025): This paper introduces a method for uncertainty quantification in deep operator networks (DeepONets). It uses predictive uncertainty estimates calibrated to model errors observed during training. The framework uses a single network, unlike ensemble approaches, and adds minimal overhead during training and inference. The authors demonstrate the model's ability to accurately reproduce solutions to partial differential equations (PDEs) and show that predictive uncertainties align well with observed error distributions. They also demonstrate the use of predictive uncertainties within an active learning framework to improve accuracy and data efficiency.

2.  **Robust Losses for Decision-Focused Learning** by Schutte et al. (2024): This paper addresses the challenge of uncertain parameters in optimization models for discrete decisions. The authors evaluate the effect of aleatoric and epistemic uncertainty on the accuracy of empirical regret as a surrogate loss. They propose three novel loss functions that approximate expected regret more robustly, improving test-sample empirical regret while maintaining equivalent computational time.

3.  **Risk-Averse Offline Reinforcement Learning** by Armengol Urpí et al. (2021): This paper introduces the Offline Risk-Averse Actor-Critic (O-RAAC) algorithm, which learns risk-averse policies in a fully offline setting. The algorithm optimizes a risk-averse criterion, namely the CVaR (Conditional Value at Risk). The authors show that O-RAAC learns policies with higher CVaR than risk-neutral approaches and guarantees distributional robustness of the average performance with respect to particular distribution shifts.

## Example Usage 2
Search papers related to a topic and specify the amount of documents. 

In [41]:
from IPython.display import display, Markdown, Latex

user_message = "I am looking for papers 6 that investigate robotics and machine learning."

chatbot = RAG_Scientific_chatbot()
answer = chatbot.chat(user_message)
display(Markdown(answer))

 - CALL: create_embedding(Robotics, machine le...)
 - CALL: search_embedded_documents(n = 6)
 - CALL: retrieve_documents(IDS = [7594, 8488, 5513, 7069, 1494, 8950])


Here are 6 papers that investigate robotics and machine learning:

1.  **Deep Learning for Image and Point Cloud Fusion in Autonomous Driving: A Review** by Cui et al. (2021) reviews recent deep-learning-based data fusion approaches that leverage both image and point cloud data. The paper gives an overview of deep learning on image and point cloud data processing, camera-LiDAR fusion methods in depth completion, object detection, semantic segmentation, tracking and online cross-sensor calibration. The authors compare these methods on publicly available datasets and identify gaps and over-looked challenges between current academic researches and real-world applications.
2.  **Smart systems, the fourth industrial revolution and new challenges in distributed computing** by El Baz and Zhu (2019) addresses smart systems and the smart world concept in the framework of the fourth industrial revolution, considers new challenges in distributed autonomous robots and computing, and illustrates a new kind of smart and reconfigurable distributed modular robot system.
3.  **The Limits and Potentials of Deep Learning for Robotics** by Sünderhauf et al. (2018) discusses robotics-specific learning, reasoning, and embodiment challenges for deep learning. The authors explain the need for better evaluation metrics, highlight the importance and unique challenges for deep robotic learning in simulation, and explore the spectrum between purely data-driven and model-driven approaches.
4.  **Online Constrained Model-based Reinforcement Learning** by Van Niekerk et al. (2017) proposes a model based approach that combines Gaussian Process regression and Receding Horizon Control to handle continuous state and action spaces while remaining within a limited time and resource budget for safe operation in robotic systems.
5.  **Shared Cross-Modal Trajectory Prediction for Autonomous Driving** by Choi et al. (2021) proposes a Cross-Modal Embedding framework that aims to benefit from the use of multiple input modalities to predict future trajectories of traffic agents in highly interactive environments. At training time, their model learns to embed a set of complementary features in a shared latent space by jointly optimizing the objective functions across different types of input data.
6.  **Trajectory-Optimized Sensing for Active Search of Tissue Abnormalities in Robotic Surgery** by Salman et al. (2018) develops an approach for guiding robots to automatically localize and find the shapes of tumors and other stiff inclusions present in the anatomy using Gaussian processes to model the stiffness distribution and active learning to direct the palpation path of the robot.


# Limitations
If we search for x, y and z papers from three topics, the RAG cannot process this information. This is where an agent would be of great use, because it can call multiple times different functions and gather all the data one by one to provide an answer.

Because of this, an agent was partially developed but the automatic calling of functions did not work correctly. In the appendix, partial code for this agent can be found.

Furthermore, it cannot open documents in other format than PDF. This is also a field for improvement.

## Appendix: Partial code for an AI Agent 

This code is provided for future improvement, given that the implementation did not work.

In [13]:
# === Tool declarations ===
create_embedding_tool = {
    "name" : "create_embedding",
    "description" : "For a given text, generate the corresponding vector embedding.",
    "parameters" : {
        "type": "OBJECT",
        "properties": {
            "text": {
                "type": "STRING",
                "description": "The input text to embed."
            }
        },
        "required": ["text"]
    }
}

search_embedded_documents_tool = {
    "name" : "search_embedded_documents",
    "description" : "Search for similar documents using a query embedding.",
    "parameters" : {
        "type": "OBJECT",
        "properties": {
            "query_embedding": {
                "type": "ARRAY",
                "items": {
                    "type": "NUMBER"  # O "INTEGER" si tus vectores son int (normalmente son floats)
                },
                "description": "The vector embedding of the input query."
            },
            "n": {
                "type": "INTEGER",
                "description": "Number of top similar documents to retrieve."
            }
        },
        "required": ["query_embedding", "n"]
    }
}

retrieve_documents_tool = {
    "name" : "retrieve_documents",
    "description" : "Retrieve detailed information about a document using its ID.",
    "parameters" : {
        "type": "OBJECT",
        "properties": {
            "doc_id": {
                "type": "INTEGER",
                "description": "The ID of the paper/document."
            }
        },
        "required": ["doc_id"]
    }
}



In [14]:
#orchestration_tools  = types.Tool(function_declarations=[create_embedding_tool, search_embedded_documents_tool])

orchestration_tools  = types.Tool(function_declarations=[create_embedding_tool, search_embedded_documents_tool])


instruction = """You are a helpful chatbot that can interact with a database of vector embeddings
of scientific papers and a database with the papers extended information. You will take the users questions and documents andgenerate

Use the following tools:
    - create_embedding(text) to convert text into vector embeddings 
    - search_embedded_documents(query_embedding, n) to obtain n papers that are similar to the embedded query 
from the database of vector embeddings.

"""

In [15]:
#tool_call = response.candidates[0].content.parts[0].function_call

#if tool_call.name == "create_embedding":
#    result = create_embedding(**tool_call.args)

#function_response_part = types.Part.from_function_response(
#    name=tool_call.name,
#    response={"result": result},
#)

#contents.append(types.Content(role="model", parts=[types.Part(function_call=tool_call)])) # Append the model's function call message
#contents.append(types.Content(role="user", parts=[function_response_part])) # Append the function response
#response = client.models.generate_content(
#    model="gemini-2.0-flash", 
#    config=types.GenerateContentConfig(
#        system_instruction=instruction,
#        tools=[orchestration_tools]
#    ),
#    contents = contents
#)
#print(response)